CASE-A

In [ ]:
#CASE-A

import numpy as np
from operator import attrgetter
import copy, random

class Particle:
    def __init__(self, solution, cost): 
        self.velocity = []
        self.solution = solution 
        self.pbest = solution 
        self.cost_pbest_solution = cost 
        self.cost_current_solution = cost 

    def assignParticleBest(self, new_pbest):
        self.pbest = new_pbest

    def returnParticleBest(self):
        return self.pbest

    def assignSolution(self, solution):
        self.solution = solution

    def returnSolution(self):
        return self.solution

    def assignParticleVelocity(self, new_velocity): 
        self.velocity = new_velocity

    def assignPbestCost(self, cost):
        self.cost_pbest_solution = cost

    def returnPbestCost(self):
        return self.cost_pbest_solution

    def assignSolutionCost(self, cost):
        self.cost_current_solution = cost

    def initialiseVelocity(self):
        del self.velocity[:]


# PSO Algorithm
class AlgoPSO:
    def __init__(self, graph, iterations, size_population, alfa=1, beta=1):
        self.iterations = iterations 
        self.graph = graph 
        self.size_population = size_population 
        self.alfa = alfa 
        self.beta = beta 
        self.particles = [] 

        solutions = self.graph.generatePaths(self.size_population)

        for d in range(len(city_names)):
          print(city_names[d],d, end='  ')

        print()

        for solution in solutions:
            particle = Particle(solution=solution, cost=graph.returnCost(solution))
            self.particles.append(particle)

    def assignGlobalBest(self, new_gbest):
        self.gbest = new_gbest

    def returnGlobalBest(self):
        return self.gbest

    def execute(self):
        for t in range(self.iterations):
            self.gbest = min(self.particles, key=attrgetter("cost_pbest_solution")) 

            for particle in self.particles:
                particle.initialiseVelocity() 
                temp_velocity = []
                solution_gbest = copy.copy(self.gbest.returnParticleBest()) 
                solution_pbest = particle.returnParticleBest()[:] 
                
                while True:
                    solution_particle = particle.returnSolution()[:] 
                    for i in range(self.graph.amount_vertices):
                        if solution_particle[i] != solution_pbest[i]:
                            swap_operator = (i, solution_pbest.index(solution_particle[i]), self.alfa)
                            temp_velocity.append(swap_operator)
                            temp = solution_pbest[swap_operator[0]]
                            solution_pbest[swap_operator[0]] = solution_pbest[swap_operator[1]]
                            solution_pbest[swap_operator[1]] = temp

                    for i in range(self.graph.amount_vertices):
                        if solution_particle[i] != solution_gbest[i]:
                            swap_operator = (i, solution_gbest.index(solution_particle[i]), self.beta)

                            temp_velocity.append(swap_operator)
                            temp = solution_gbest[swap_operator[0]]
                            solution_gbest[swap_operator[0]] = solution_gbest[swap_operator[1]]
                            solution_gbest[swap_operator[1]] = temp

                    particle.assignParticleVelocity(temp_velocity)

                    for swap_operator in temp_velocity:
                        if random.random() <= swap_operator[2]: 
                            temp = solution_particle[swap_operator[0]]
                            solution_particle[swap_operator[0]] = solution_particle[swap_operator[1]]
                            solution_particle[swap_operator[1]] = temp
                    if satisfy_color_order(solution_particle,color):
                        break
                particle.assignSolution(solution_particle) 
                cost_current_solution = self.graph.returnCost(solution_particle)
                particle.assignSolutionCost(cost_current_solution) 

                if cost_current_solution < particle.returnPbestCost():
                    particle.assignParticleBest(solution_particle)
                    particle.assignPbestCost(cost_current_solution)
            p = self.returnGlobalBest().returnParticleBest()
            print("Iteration: ",t,"| Best Path: ",p,"| Cost: ", graph.returnCost(p))


class TSPGraph:
    def __init__(self, number_vertices): 
        self.amount_vertices = number_vertices 
        self.edges = {} 
        self.vertices = set() 

    def returnCost(self, path):
        sum_dist = 0
        for i in range(self.amount_vertices - 1):
            sum_dist += self.edges[(path[i], path[i+1])]
        return sum_dist

    def alreadyEdge(self, left, right):
        if (left, right) in self.edges:
            return True
        else:
            return False
        
    def addEdge(self, left, right, cost=0):
        if not self.alreadyEdge(left, right):
            self.edges[(left, right)] = cost
            self.vertices.add(left)
            self.vertices.add(right)

    def returnAllVertices(self):
        return list(self.vertices)

    def generatePaths(self, max_size):
        random_paths = []
        list_vertices = list(self.vertices)

        for i in range(max_size):
            while True:
                list_temp = list_vertices[:] 
                random.shuffle(list_temp) 
                if list_temp not in random_paths and  satisfy_color_order(list_temp, color):
                    random_paths.append(list_temp)
                    break 

        return random_paths

def satisfy_color_order(path, color):
    red_flag = False
    yellow_flag = False
    for i in range(len(path)):
        if color[path[i]] == "red":
            red_flag = True
        elif color[path[i]] == "yellow" and red_flag == False:
            yellow_flag = True

        elif color[path[i]] == "yellow" and red_flag == True:
            return False
        elif color[path[i]] == "green" and (red_flag == True or yellow_flag == True):
            return False
        
        elif color[path[i]] == "white" and (red_flag == True or yellow_flag == True):
            yellow_flag = False
            red_flag = False
            
    return True

cities = [[0, 169, 111, 227, 263, 299, 328, 289, 148],
[169, 0, 231, 175, 356, 268, 215, 359, 310],
[111, 231, 0, 282, 150, 372, 363, 397, 255],
[227, 175, 282, 0, 444, 92, 103, 189, 379],
[263, 356, 150, 444, 0,	538, 523, 553, 412],
[299, 268, 372, 92, 538, 0, 139, 226, 415],
[328, 215, 363, 103, 523, 139, 0, 293, 426],
[289, 359, 397, 189, 553, 226, 293, 0, 308],
[148, 310, 255, 379, 412, 415,	426, 308, 0]]

#Holds color status of each city.
color = ["white","green","green","red","yellow","yellow","white","green","green"]
city_names = ['Ahmedabad', 'Bhavnagar', 'Vadodara', 'Rajkot', 'Surat', 'Jamnagar', 'Junagadh', 'Gandhidham', 'Palanpur']
graph = TSPGraph(number_vertices=9)

#Creating graph from cities array.
for i in range(len(cities)):
    for j in range(len(cities)):
        graph.addEdge(i, j, cities[i][j])

pso = AlgoPSO(graph, iterations=100, size_population=150, alfa=0.8, beta=0.8)
pso.execute()

print('\nCase-A: Final Best Path: %s | Cost: %d\n' % (pso.returnGlobalBest().returnParticleBest(), pso.returnGlobalBest().returnPbestCost()))
for d in range(len(city_names)):
  print(city_names[d],d, end='  ')

Ahmedabad 0  Bhavnagar 1  Vadodara 2  Rajkot 3  Surat 4  Jamnagar 5  Junagadh 6  Gandhidham 7  Palanpur 8  
Iteration:  0 | Best Path:  [1, 5, 6, 3, 0, 7, 8, 2, 4] | Cost:  1739
Iteration:  1 | Best Path:  [5, 3, 6, 1, 0, 7, 8, 2, 4] | Cost:  1581
Iteration:  2 | Best Path:  [7, 5, 3, 6, 1, 0, 8, 2, 4] | Cost:  1358
Iteration:  3 | Best Path:  [7, 5, 3, 6, 1, 0, 8, 2, 4] | Cost:  1358
Iteration:  4 | Best Path:  [7, 5, 3, 6, 1, 8, 0, 2, 4] | Cost:  1355
Iteration:  5 | Best Path:  [7, 5, 3, 6, 1, 8, 0, 2, 4] | Cost:  1355
Iteration:  6 | Best Path:  [7, 5, 3, 6, 1, 8, 0, 2, 4] | Cost:  1355
Iteration:  7 | Best Path:  [7, 5, 3, 6, 1, 8, 0, 2, 4] | Cost:  1355
Iteration:  8 | Best Path:  [7, 5, 3, 6, 1, 8, 0, 2, 4] | Cost:  1355
Iteration:  9 | Best Path:  [7, 5, 3, 6, 1, 8, 0, 2, 4] | Cost:  1355
Iteration:  10 | Best Path:  [7, 5, 3, 6, 1, 8, 0, 2, 4] | Cost:  1355
Iteration:  11 | Best Path:  [7, 5, 3, 6, 1, 8, 0, 2, 4] | Cost:  1355
Iteration:  12 | Best Path:  [7, 5, 3, 6, 1, 8, 0,

CASE-B

In [ ]:
#CASE-B

'''
Ahmedabad(Node-0) city is allowed to be visited twice in this case. This is decided based on the fact that the sum of distances 
of all incoming routes to this city is lower, i.e there exists a path from city 'A' to city 'B' through Ahmedabad shorter than 
all other paths from city 'A' to city 'B' through any other city other than Ahmedabad. Also, the color status of Ahmedabad is 'white' 
and therefore much safer than 'red' or 'yellow' coloured cities. Allowing to visit a white color city more than once would avoid the need to visit more covid prone 'red' and 
'yellow' cities more than once if the need arises. This way, the traveller is more safer even though he/she has to travel a few extra miles. (7 miles here compared to case-A).
'''
import numpy as np
from operator import attrgetter
import copy, random

class Particle:
    def __init__(self, solution, cost): 
        self.solution = solution 
        self.pbest = solution 
        self.cost_current_solution = cost 
        self.cost_pbest_solution = cost 
        self.velocity = []

    # setters and getters
    def assignParticleBest(self, new_pbest):
        self.pbest = new_pbest

    def returnParticleBest(self):
        return self.pbest

    def assignParticleVelocity(self, new_velocity): 
        self.velocity = new_velocity

    def assignSolution(self, solution):
        self.solution = solution

    def returnSolution(self):
        return self.solution

    def assignPbestCost(self, cost):
        self.cost_pbest_solution = cost

    def returnPbestCost(self):
        return self.cost_pbest_solution

    def assignSolutionCost(self, cost):
        self.cost_current_solution = cost

    def initialiseVelocity(self):
        del self.velocity[:]


# PSO Algorithm
class AlgoPSO:
    def __init__(self, graph, iterations, size_population, alfa=1, beta=1):
        self.iterations = iterations 
        self.size_population = size_population 
        self.graph = graph
        self.alfa = alfa 
        self.beta = beta 
        self.particles = []

        #Initialisation.
        solutions = self.graph.generatePaths(self.size_population)
        
        for d in range(len(city_names)):
            print(city_names[d],d, end='  ')
        print()
        for solution in solutions:
            particle = Particle(solution=solution, cost=graph.returnCost(solution))
            self.particles.append(particle)

    def assignGlobalBest(self, new_gbest):
        self.gbest = new_gbest

    def returnGlobalBest(self):
        return self.gbest

    def execute(self):
        for t in range(self.iterations):
            self.gbest = min(self.particles, key=attrgetter("cost_pbest_solution"))

            for particle in self.particles:
                particle.initialiseVelocity() 
                temp_velocity = []
                solution_gbest = copy.copy(self.gbest.returnParticleBest())
                solution_pbest = particle.returnParticleBest()[:]
                
                while True:
                    solution_particle = particle.returnSolution()[:] 
                    for i in range(self.graph.amount_vertices):
                        if solution_particle[i] != solution_pbest[i]:
                            swap_operator = (i, solution_pbest.index(solution_particle[i]), self.alfa)
                            temp_velocity.append(swap_operator)

                            # Swap
                            temp = solution_pbest[swap_operator[0]]
                            solution_pbest[swap_operator[0]] = solution_pbest[swap_operator[1]]
                            solution_pbest[swap_operator[1]] = temp

                    for i in range(self.graph.amount_vertices):
                        if solution_particle[i] != solution_gbest[i]:
                            swap_operator = (i, solution_gbest.index(solution_particle[i]), self.beta)
                            temp_velocity.append(swap_operator)

                            # Swap
                            temp = solution_gbest[swap_operator[0]]
                            solution_gbest[swap_operator[0]] = solution_gbest[swap_operator[1]]
                            solution_gbest[swap_operator[1]] = temp

                    particle.assignParticleVelocity(temp_velocity)

                    for swap_operator in temp_velocity:
                        if random.random() <= swap_operator[2]: 
                            # Swap
                            temp = solution_particle[swap_operator[0]]
                            solution_particle[swap_operator[0]] = solution_particle[swap_operator[1]]
                            solution_particle[swap_operator[1]] = temp
                    indexes = np.where(np.array(solution_particle) == 0)[0]
                    if satisfy_color_order(solution_particle,color) and abs(indexes[0]-indexes[1])>1:
                        break
                particle.assignSolution(solution_particle) 
                cost_current_solution = self.graph.returnCost(solution_particle)
                particle.assignSolutionCost(cost_current_solution) 

                if cost_current_solution < particle.returnPbestCost():
                    particle.assignParticleBest(solution_particle)
                    particle.assignPbestCost(cost_current_solution)
            p = self.returnGlobalBest().returnParticleBest()
            print("Iteration: ",t,"| Best Path: ",p,"| Cost: ", graph.returnCost(p))

class TSPGraph:
    def __init__(self, number_vertices): 
        self.edges = {} 
        self.vertices = set() 
        self.amount_vertices = number_vertices

    def returnCost(self, path):
        total_cost = 0
        for i in range(self.amount_vertices - 1):
            total_cost += self.edges[(path[i], path[i+1])]
        return total_cost

    def alreadyEdge(self, left, right):
        return True if (left, right) in self.edges else False

    def addEdge(self, left, right, cost=0):
        if not self.alreadyEdge(left, right):
            self.edges[(left, right)] = cost
            self.vertices.add(left)
            self.vertices.add(right)

    def returnAllVertices(self):
        return list(self.vertices)

    def generatePaths(self, max_size):

        random_paths = []
        list_vertices = list(self.vertices)

        #Ahmedabad is chosen as the city to be visited twice, so add an extra node for Ahmedabad(node-0).
        list_vertices.append(0)

        for i in range(max_size):
            while True:
                list_temp = list_vertices[:] 
                random.shuffle(list_temp)

                #Get indexes of all occurences of Ahmedabad.
                indexes = np.where(np.array(list_temp) == 0)[0]

                #Check if generated path satisfies color restrictions and that same city does not occur adjacently.
                if list_temp not in random_paths and satisfy_color_order(list_temp, color) and abs(indexes[0]-indexes[1])>1:
                    random_paths.append(list_temp)
                    break 

        return random_paths

def satisfy_color_order(path, color):
    red_flag = False
    yellow_flag = False
    for i in range(len(path)):
        if color[path[i]] == "red":
            red_flag = True
        elif color[path[i]] == "yellow" and red_flag == False:
            yellow_flag = True

        elif color[path[i]] == "yellow" and red_flag == True:
            return False
        elif color[path[i]] == "green" and (red_flag == True or yellow_flag == True):
            return False
        
        elif color[path[i]] == "white" and (red_flag == True or yellow_flag == True):
            yellow_flag = False
            red_flag = False
            
    return True

graphofcities = [[0, 169, 111, 227, 263, 299, 328, 289, 148],
[169, 0, 231, 175, 356, 268, 215, 359, 310],
[111, 231, 0, 282, 150, 372, 363, 397, 255],
[227, 175, 282, 0, 444, 92, 103, 189, 379],
[263, 356, 150, 444, 0,	538, 523, 553, 412],
[299, 268, 372, 92, 538, 0, 139, 226, 415],
[328, 215, 363, 103, 523, 139, 0, 293, 426],
[289, 359, 397, 189, 553, 226, 293, 0, 308],
[148, 310, 255, 379, 412, 415,	426, 308, 0]]

color = ["white","green","green","red","yellow","yellow","white","green","green"]
city_names = ['Ahmedabad', 'Bhavnagar', 'Vadodara', 'Rajkot', 'Surat', 'Jamnagar', 'Junagadh', 'Gandhidham', 'Palanpur']
graph = TSPGraph(number_vertices=10)

for i in range(len(graphofcities)):
    for j in range(len(graphofcities)):
        graph.addEdge(i, j, graphofcities[i][j])

pso = AlgoPSO(graph, iterations=100, size_population=150, alfa=0.8, beta=0.8)
pso.execute()

print('\nCase-B: Final Best Path: %s | Cost: %d\n' % (pso.returnGlobalBest().returnParticleBest(), pso.returnGlobalBest().returnPbestCost()))
for d in range(len(city_names)):
  print(city_names[d],d, end='  ')

Ahmedabad 0  Bhavnagar 1  Vadodara 2  Rajkot 3  Surat 4  Jamnagar 5  Junagadh 6  Gandhidham 7  Palanpur 8  
Iteration:  0 | Best Path:  [4, 0, 8, 2, 1, 0, 7, 5, 3, 6] | Cost:  1776
Iteration:  1 | Best Path:  [1, 7, 5, 3, 6, 0, 8, 0, 2, 4] | Cost:  1665
Iteration:  2 | Best Path:  [1, 7, 5, 3, 6, 0, 8, 0, 2, 4] | Cost:  1665
Iteration:  3 | Best Path:  [1, 7, 5, 3, 6, 0, 8, 0, 2, 4] | Cost:  1665
Iteration:  4 | Best Path:  [0, 1, 5, 3, 6, 7, 8, 0, 2, 4] | Cost:  1642
Iteration:  5 | Best Path:  [7, 3, 6, 1, 5, 0, 8, 0, 2, 4] | Cost:  1631
Iteration:  6 | Best Path:  [7, 5, 6, 3, 0, 1, 8, 0, 2, 4] | Cost:  1583
Iteration:  7 | Best Path:  [7, 5, 6, 3, 0, 1, 8, 0, 2, 4] | Cost:  1583
Iteration:  8 | Best Path:  [7, 5, 6, 3, 0, 1, 8, 0, 2, 4] | Cost:  1583
Iteration:  9 | Best Path:  [7, 5, 6, 1, 3, 0, 8, 0, 2, 4] | Cost:  1539
Iteration:  10 | Best Path:  [7, 5, 3, 6, 1, 0, 8, 0, 2, 4] | Cost:  1362
Iteration:  11 | Best Path:  [7, 5, 3, 6, 1, 0, 8, 0, 2, 4] | Cost:  1362
Iteration:  12